## EventBridge Workbook

[Amazon EventBridge](https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-get-started.html) is a serverless service that enables us to create *rules* that route *events* to *target* processes.   

For the Input Management use case, the *event* occurs when a file arrives in a particular 'folder' on the landing pad.  File-arrived events trigger one or more *target* processes.  For example, ingesting the file into the Data Lake, kicking off batch processing using the file as input, or forwarding the file elsewhere with a new name.  

This workbook contains an example of how to Create/Update a Rule and its Target.  This Rule is actually created by [CloudFormation template](ci-cd/cfn_template.yaml), but this Python example is provided to illustrate how basic parameters are used to build and apply naming conventions across the resource stack.  Ultimately, we may find it easier to manage EB Rules in script, since Console and CloudFormation are 'fiddly' for this. 


In [7]:
# Initialize Persistent %store & Notebook Globals
import json
import boto3
import os

%store -r 

S_region = 'us-east-1'

S_sys_abbrev = "FSDATA"
%store S_sys_abbrev S_region

%store 

os.environ['AWS_DEFAULT_REGION'] = S_region


Stored 'S_sys_abbrev' (str)
Stored 'S_region' (str)
Stored variables and their in-db values:
S_account_id               -> '{aws_acct}'
S_arn_template             -> 'arn:aws:$Service:us-east-2:{aws_acct}:$Resource
S_code_bucket              -> 'daab-lab-code-us-east-1'
S_partition                -> 'aws'
S_region                   -> 'us-east-1'
S_rootdir                  -> '/home/ec2-user/SageMaker/daab-simple'
S_stack                    -> 'daab-lab-smpl-main'
S_sys_abbrev               -> 'FSDATA'


In [9]:
# STEP 1 -- Create/Update EventBridge Rule for 'File Arrived' Event
eb_client = boto3.client('events')

# Define the Bucket and File Name Prefix for the arrived file ...
#landing_pad_bucket = f"{S_stack}_landing_pad"
#sys_abbrev = "FSDATA"         # e.g., CRS, CSNG, TOP, etc.
file_prefix = "FDMD.FSDATA"   # First 2 nodes of the dataset name (i.e., Qualifier & Base Name)

eb_event_pattern = {
  "detail-type": ["Object Created"],
  "source": ["aws.s3"],
  "detail": {
    "bucket": {
      "name": [ S_landing_pad_bucket ]
    },
    "object": {
      "key": [{
        "prefix": f"{S_sys_abbrev}/Inbound/{file_prefix}"
      }]
    }
  }
}

eb_rulename = f"{S_stack}-{file_prefix}"
response = eb_client.put_rule(
    Name= eb_rulename,
    EventPattern=json.dumps(eb_event_pattern),
    Description= f"When '{file_prefix}*' files arrive in 's3://{S_landing_pad_bucket}/{S_sys_abbrev}/Inbound/' ..."
)

print(f"Put Rule '{eb_rulename}' ")


Put Rule 'daab-lab-fp5a-FDMD.FSDATA' 


In [11]:
# STEP 2 -- Create Target(s) to Process the 'File Arrived' Event

# ARN of the Lambda function
initiator_arn = S_arn_template.replace('$Service','lambda').replace('$Resource', f"function:{S_stack}-Process_Initiator")

# ARN of the Step Function initiated by Lambda 
step_fn_arn   = S_arn_template.replace('$Service','states').replace('$Resource', f"stateMachine:{S_stack}-Extract_Zip_to_Parquet")

# Define Target with customized InputTemplate passed to Lambda (and onward to Step Function)
eb_rule_target = {
    "Id": "target0",
    "Arn": initiator_arn,
    "InputTransformer": {
        "InputPathsMap": {
            "detail-bucket-name": "$.detail.bucket.name",
            "detail-object-key": "$.detail.object.key",
            "source": "$.source"
        },
        "InputTemplate": json.dumps ({
            "source": "<source>",
            "detail": {
                "bucket": {
                "name": "<detail-bucket-name>"
                },
                "object": { 
                    "key": "<detail-object-key>"
                    }
                },
            "process_parms":  {
                "GlueDatabaseName": f"{S_stack}-{S_sys_abbrev}",

                "S3LandingPadBucket" : f"{S_stack}-landing-pad",
                "S3LandingPadInput"  : f"{S_sys_abbrev}/Inbound",
                "S3LandingPadOutput" : f"{S_sys_abbrev}/Outbound",

                "S3DatalakeBucket": f"{S_stack}-datalake",
                "S3DatalakeInput" : "n/a",
                "S3DatalakeOutput": f"{S_sys_abbrev}/PARQUET",

                "StepFnArn" : step_fn_arn
            } 
        } )
    }
}
response = eb_client.put_targets(
    Rule = eb_rulename,
    Targets = [ eb_rule_target ]
)
print(f"Put Target(s) for Rule '{eb_rulename}' ")

Put Target(s) for Rule 'daab-lab-fp5a-FDMD.FSDATA' 


In [11]:
eb_client = boto3.client('events')

eb_rulename = 'daab-lab-fp5a-FDMD.FSDATA'



response = eb_client.describe_rule( Name = eb_rulename )

print(json.dumps(response, indent=2))




{
  "Name": "daab-lab-fp5a-FDMD.FSDATA",
  "Arn": "arn:aws:events:us-east-1:{aws_acct}:rule/daab-lab-fp5a-FDMD.FSDATA",
  "EventPattern": "{\"detail-type\": [\"Object Created\"], \"source\": [\"aws.s3\"], \"detail\": {\"bucket\": {\"name\": [\"daab-lab-fp5a-landing-pad\"]}, \"object\": {\"key\": [{\"prefix\": \"FSDATA/Inbound/FDMD.FSDATA\"}]}}}",
  "State": "ENABLED",
  "Description": "When 'FDMD.FSDATA*' files arrive in 's3://daab-lab-fp5a-landing-pad/FSDATA/Inbound/' ...",
  "EventBusName": "default",
  "CreatedBy": "{aws_acct}",
  "ResponseMetadata": {
    "RequestId": "b2b72c81-36c5-40bd-b0a6-1782c9dde1d0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "b2b72c81-36c5-40bd-b0a6-1782c9dde1d0",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "513",
      "date": "Wed, 11 Jan 2023 20:24:40 GMT"
    },
    "RetryAttempts": 0
  }
}


In [13]:
response = eb_client.list_targets_by_rule(
    Rule=eb_rulename,
    #EventBusName='string',
    #NextToken='string',
    #Limit=123
)

print(json.dumps(response, indent=2))


{
  "Targets": [
    {
      "Id": "target0",
      "Arn": "arn:aws:lambda:us-east-1:{aws_acct}:function:daab-lab-fp5a-Process_Initiator",
      "InputTransformer": {
        "InputPathsMap": {
          "detail-bucket-name": "$.detail.bucket.name",
          "detail-object-key": "$.detail.object.key",
          "source": "$.source"
        },
        "InputTemplate": "{\"source\": \"<source>\", \"detail\": {\"bucket\": {\"name\": \"<detail-bucket-name>\"}, \"object\": {\"key\": \"<detail-object-key>\"}}, \"process_parms\": {\"GlueDatabaseName\": \"daab-lab-fp5a-FSDATA\", \"S3LandingPadBucket\": \"daab-lab-fp5a-landing-pad\", \"S3LandingPadInput\": \"FSDATA/Inbound\", \"S3LandingPadOutput\": \"FSDATA/Outbound\", \"S3DatalakeBucket\": \"daab-lab-fp5a-datalake\", \"S3DatalakeInput\": \"n/a\", \"S3DatalakeOutput\": \"FSDATA/PARQUET\", \"StepFnArn\": \"arn:aws:states:us-east-1:{aws_acct}:stateMachine:daab-lab-fp5a-Extract_Zip_to_Parquet\"}}"
      }
    }
  ],
  "ResponseMetadata": {
    "

In [5]:
lambda_client = boto3.client('lambda')


fn_name = 'arn:aws:lambda:us-east-1:{aws_acct}:function:daab-lab-fp5a-Process_Initiator'
response = lambda_client.get_function( FunctionName = fn_name )

print( json.dumps(response, indent=2))
                            

{
  "ResponseMetadata": {
    "RequestId": "aa82408e-edec-41b8-bff9-5869f2595576",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 08 Jan 2023 14:57:36 GMT",
      "content-type": "application/json",
      "content-length": "3340",
      "connection": "keep-alive",
      "x-amzn-requestid": "aa82408e-edec-41b8-bff9-5869f2595576"
    },
    "RetryAttempts": 0
  },
  "Configuration": {
    "FunctionName": "daab-lab-fp5a-Process_Initiator",
    "FunctionArn": "arn:aws:lambda:us-east-1:{aws_acct}:function:daab-lab-fp5a-Process_Initiator",
    "Runtime": "python3.9",
    "Role": "arn:aws:iam::{aws_acct}:role/daab-lab-fp5a-Process_Initiator_Role",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 1734,
    "Description": "Triggered by EventBridge to Execute a Process ARN Specified in the Target Input",
    "Timeout": 60,
    "MemorySize": 128,
    "LastModified": "2023-01-08T14:48:45.553+0000",
    "CodeSha256": "pPYqmApIjK9oVyGI6nz5PGg4Dmmu1KkPZ0MS22wPUZE=

In [6]:
response = lambda_client.get_policy( FunctionName = fn_name )

print( json.dumps(response, indent=2))


{
  "ResponseMetadata": {
    "RequestId": "f59f1b49-5198-433d-ba59-09621d87f71a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 08 Jan 2023 14:59:53 GMT",
      "content-type": "application/json",
      "content-length": "514",
      "connection": "keep-alive",
      "x-amzn-requestid": "f59f1b49-5198-433d-ba59-09621d87f71a"
    },
    "RetryAttempts": 0
  },
  "Policy": "{\"Version\":\"2012-10-17\",\"Id\":\"default\",\"Statement\":[{\"Sid\":\"lambda-5ffd2641-a474-4b78-9472-93d217f4ab9d\",\"Effect\":\"Allow\",\"Principal\":{\"Service\":\"events.amazonaws.com\"},\"Action\":\"lambda:InvokeFunction\",\"Resource\":\"arn:aws:lambda:us-east-1:{aws_acct}:function:daab-lab-fp5a-Process_Initiator\",\"Condition\":{\"ArnLike\":{\"AWS:SourceArn\":\"arn:aws:events:us-east-1:{aws_acct}:rule/daab-lab-fp5a-FDMD.FSDATA\"}}}]}",
  "RevisionId": "bd6510dd-43ba-4855-bb1a-1384c0d911e0"
}
